In [ ]:
import os
import posixpath
import xarray as xr
import rioxarray
import numpy as np
import matplotlib.pyplot as plt
from skimage import filters, img_as_float
from dask_image.ndfilters import convolve as dask_convolve
from scipy.ndimage import convolve

# parallel computing
from dask.distributed import Client

# to keep xarray quiet
# import warnings
# warnings.filterwarnings("ignore")

In [ ]:
def set_env():
    os.environ["GS_NO_SIGN_REQUEST"] = "YES"

set_env()

client = Client(n_workers=1, threads_per_worker=4)
client.run(set_env)
client

In [ ]:
images_root = 'D:/OneDrive/Documents/Cours/4A/SFE/data/KH-5_ARGON_images'
products = []

for x in os.listdir(images_root):
    if os.path.isdir(posixpath.join(images_root, x)):
        products.append(x)
        
products

In [ ]:
file = posixpath.join(images_root, "DS09034A008MC021", "DS09034A008MC021_a_geo.tif")
raster = rioxarray.open_rasterio(file, chunks=True)
raster

In [ ]:
shx, shy = raster.shape[1:]
shx, shy

In [ ]:
raster_rs = raster[0, 0:shy:10, 0:shx:10]
raster_zoom = raster[0, 1400*10:1600*10, 1600*10:1800*10]

fig, axs = plt.subplots(ncols=2, figsize=(15, 7))
axs[0].imshow(raster_rs, cmap='gray')
axs[1].imshow(raster_zoom, cmap='gray')

In [ ]:
# raster_rs = raster[0, 0:shy:10, 0:shx:10].compute()
# print(raster_rs.shape)

# fig, axs = plt.subplots(ncols=2, figsize=(15, 7))
# axs[0].imshow(raster_rs, cmap='gray')
# axs[1].imshow(raster_rs[550:850, 900:1200], cmap='gray')

In [ ]:
# Define emboss kernel
emboss_kernel = np.zeros((3, 3))
emboss_kernel[0, 0] = -1
emboss_kernel[2, 2] = 1

embossed_rs = convolve(raster_rs.astype(float).values, emboss_kernel)
embossed_zoom = convolve(raster_zoom.astype(float).values, emboss_kernel)

fig, axs = plt.subplots(ncols=2, figsize=(15, 7))
axs[0].imshow(embossed_rs, cmap='gray')
axs[1].imshow(embossed_zoom, cmap='gray')

In [ ]:
# same on full raster
raster_em = dask_convolve(raster.astype(float).values, emboss_kernel)